In [1]:
import sys
import pandas as pd

sys.path.append('../')
import pyretro.boxball_loader as bbl

df = pd.read_parquet(f'{bbl.RETROSHEET_DIR}/daily.parquet')
df.shape

In [3]:
# Drop duplicates (there are hundreds of thousands of dups in the daily table)
dailies = df.sort_values(by=['game_id', 'player_id', 'b_pa']).drop_duplicates(subset=['game_id', 'player_id', 'team_id', 'slot_ct'], keep='first')

df.shape, dailies.shape

((6314255, 154), (5344339, 154))

In [4]:
# Add year
dailies['yr'] = dailies['game_dt'].dt.year
dailies.shape

(5344339, 155)

In [8]:
# Add game_type and team_game_number from gl_teams
glt = bbl.load_gamelog_teams(bbl.GameType.ALL, bbl.Eras.All)[['game_id', 'team', 'game_type', 'team_game_number']].rename(columns={'team': 'team_id'})
glt.columns, glt.shape

(Index(['game_id', 'team_id', 'game_type', 'team_game_number'], dtype='object'),
 (456386, 4))

In [9]:
dailies = pd.merge(dailies, glt, on=['game_id', 'team_id'])
dailies.head()

,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,f_rf_out,f_rf_tc,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp,yr,game_type,team_game_number
0,ALS193307060,1933-07-06,0,1933-07-06,ALS,avere101,9,3,True,NLS,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1933,ASG,1
1,ALS193307060,1933-07-06,0,1933-07-06,ALS,chapb102,1,1,True,NLS,...,3.0,1.0,1.0,0.0,0.0,0,0.0,1933,ASG,1
2,ALS193307060,1933-07-06,0,1933-07-06,ALS,cronj101,7,1,True,NLS,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1933,ASG,1
3,ALS193307060,1933-07-06,0,1933-07-06,ALS,crowg102,9,2,True,NLS,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1933,ASG,1
4,ALS193307060,1933-07-06,0,1933-07-06,ALS,dykej101,6,1,True,NLS,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1933,ASG,1


In [6]:
# Write
dailies.to_parquet(f'{bbl.BASE_DATA_DIR}/mine/daily.parquet')

In [10]:
dailies.tail()

,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,f_rf_out,f_rf_tc,f_rf_po,f_rf_a,f_rf_e,f_rf_dp,f_rf_tp,yr,game_type,team_game_number
5344334,WS4187205170,1872-05-17,0,1872-05-17,WS4,hollh101,1,1,True,WS3,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,RS,7
5344335,WS4187205170,1872-05-17,0,1872-05-17,WS4,lennb101,3,1,True,WS3,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,RS,7
5344336,WS4187205170,1872-05-17,0,1872-05-17,WS4,mince101,2,1,True,WS3,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,RS,7
5344337,WS4187205170,1872-05-17,0,1872-05-17,WS4,steab101,9,1,True,WS3,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,RS,7
5344338,WS4187205170,1872-05-17,0,1872-05-17,WS4,whitw102,7,1,True,WS3,...,0.0,0.0,0.0,0.0,0.0,0,0.0,1872,RS,7
